In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from path import Path
import sklearn as skl

In [2]:
# Load file
file_path = Path('charity_data.csv')
charity_df = pd.read_csv(file_path, index_col =0)
charity_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Keep only active campaigns
charity_df = charity_df[charity_df['STATUS'] != 0]
# Save Name to a new dataframe 
name_df = pd.DataFrame(charity_df, columns=['NAME'], index=charity_df.index)
# Drop 'NAME' and "STATUS" columns
charity_df = charity_df.drop(columns=['NAME', 'STATUS'])
# Check for null values
charity_df.isnull().sum()


APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes=='object'].index.tolist()
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [5]:
# Group rare values into "Others" category
application_counts = charity_df.APPLICATION_TYPE.value_counts()
classification_counts = charity_df.CLASSIFICATION.value_counts()
# Application type count
application_counts

T3     27032
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Replace application type count < 150 with "Others"
replace_applications = application_counts[application_counts < 200].index
for application in replace_applications:
    charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].replace(application,"Others")
charity_df['APPLICATION_TYPE'].value_counts()

T3        27032
T4         1542
T6         1216
T5         1173
T19        1065
T8          737
T7          725
T10         528
Others      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# First 40 Classification count
classification_counts[:40]

C1000    17323
C2000     6073
C1200     4837
C3000     1918
C2100     1882
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
C2710        3
C1260        3
C0           3
Name: CLASSIFICATION, dtype: int64

In [8]:
# Replace Classification with value < 100 as "Others"
replace_classifications = classification_counts[classification_counts < 100].index
for classification in replace_classifications:
    charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(classification, "Others")
charity_df['CLASSIFICATION'].value_counts()

C1000     17323
C2000      6073
C1200      4837
C3000      1918
C2100      1882
C7000       777
Others      669
C1700       287
C4000       194
C5000       116
C1270       114
C2700       104
Name: CLASSIFICATION, dtype: int64

In [9]:
# OneHotEncoder
# Create intance
enc = OneHotEncoder(sparse=False)
# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]), index=charity_df.index)

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Others,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10531628,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10547893,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10553066,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10556103,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge and drop
charity_df = charity_df.merge(encode_df, how='left', left_index=True, right_index=True)
charity_df = charity_df.drop(columns=charity_cat)
charity_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Others,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10531628,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10547893,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10553066,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10556103,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split prepocessed data into features and target arrays
y = charity_df['IS_SUCCESSFUL']
X = charity_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create StandardScaler instance
X_scaler = StandardScaler()
# Fit the scaler
X_scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net
number_input_features = len(X.columns)
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

# Create Keras Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4900      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 20,101
Trainable params: 20,101
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Train on 25720 samples
Epoch 1/50
25720/25720 [==============================] - 2s 80us/sample - loss: 0.5708 - accuracy: 0.7210
Epoch 2/50
25720/25720 [==============================] - 2s 66us/sample - loss: 0.5553 - accuracy: 0.7299
Epoch 3/50
25720/25720 [==============================] - 2s 60us/sample - loss: 0.5524 - accuracy: 0.7315
Epoch 4/50
25720/25720 [==============================] - 2s 64us/sample - loss: 0.5507 - accuracy: 0.7304
Epoch 5/50
25720/25720 [==============================] - 2s 73us/sample - loss: 0.5489 - accuracy: 0.7318
Epoch 6/50
25720/25720 [==============================] - 2s 67us/sample - loss: 0.5485 - accuracy: 0.7315
Epoch 7/50
25720/25720 [==============================] - 2s 61us/sample - loss: 0.5468 - accuracy: 0.7334
Epoch 8/50
25720/25720 [==============================] - 2s 62us/sample - loss: 0.5462 - accuracy: 0.7334
Epoch 9/50
25720/25720 [==============================] - 2s 70us/sample - loss: 0.5456 - accuracy: 0.7336
Epoch 10/50
25

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

8574/8574 - 0s - loss: 0.5567 - accuracy: 0.7300
Loss: 0.5567231813677269, Accuracy: 0.7299976944923401
